# Getting started with qcodes-elab

In [1]:
## Initialising

import qcodes as qc
from qcodes.tests.instrument_mocks import DummyMeasurementInstrument

instrument=DummyMeasurementInstrument(name='instrument')

station=qc.Station(instrument)

# Tell qcodes where to save data, relative to the folder containing this notebook.
qc.set_data_folder('data')

## Setting and getting values

In [2]:
instrument.output1(0.1)

In [3]:
instrument.output1()

0.1

In [ ]:
instrument.output2(1.56)

In [ ]:
instrument.output2()

In [ ]:
instrument.input1()

In [ ]:
instrument.input2()

In [ ]:
print(instrument.input1(),instrument.input2())

## Running an experiment

In [4]:
station.set_measurement(instrument.input1,instrument.input2)

loop=qc.Loop(instrument.output1.sweep(0,1,num=101),delay=0.1).each(*station.measure())

data=loop.get_data_set(name='ImaginaryDevice Vout(0 0.1)V T=23.4K')
print(data)

DataSet:
   location = 'data/#012_ImaginaryDevice Vout(0 0.1)V T=23.4K_2022-11-28_11-57-13'
   <Type>   | <array_id>             | <array.name> | <array.shape>
   Setpoint | instrument_output1_set | output1      | (101,)
   Measured | instrument_input1      | input1       | (101,)
   Measured | instrument_input2      | input2       | (101,)
   Measured | timer                  | timer        | (101,)


In [ ]:
pp=qc.Plot()
data.publisher=pp

pp.add(data.instrument_input1,subplot=0)
pp.add(data.instrument_input2,subplot=1)
pp.add(data.timer,subplot=2)

#pp.add_multiple(data.instrument_input1,data.instrument_input2,data.timer)

_=loop.run()

## 2D loops

In [ ]:
station.set_measurement(instrument.input1,instrument.input2)

loop=qc.Loop(instrument.output1.sweep(-1,1,num=21),delay=0.01)
outerloop=qc.Loop(instrument.output2.sweep(-0.5,0.5,num=21),delay=0.1).each(loop)

data=outerloop.get_data_set(name='ImaginaryDevice Vout1(-1 1)V Vout2(-0.5 0.5) T=23.2K')
print(data)

In [ ]:
pp=qc.Plot()
data.publisher=pp
pp.add(data.instrument_input1,subplot=0)
pp.add(data.instrument_input2,subplot=1)
_=outerloop.run()

## Custom parameters

In [ ]:
current=qc.ScaledParameter(instrument.input1,gain=1e-6,name='current',label='Current',unit='A')

def get_resistance():
    return(instrument.input2()/current())
resistance=qc.Parameter(name='resistance',label='R',unit='Ohm',get_cmd=get_resistance)

def set_vector(value):
    instrument.output1(value+0.1)
    instrument.output2(value*2.5+0.8)
def get_vector():
    return(instrument.output1())
OutputVector=qc.Parameter(name='OutputVector',label='Vector',unit='V',get_cmd=get_vector,set_cmd=set_vector)

def set_stepper(val):
    0
stepper=qc.Parameter(name='stepper',label='step',unit='number',set_cmd=set_stepper)

In [ ]:
timeloop=qc.Loop(stepper.sweep(0,1,num=101),delay=0.1)
data=timeloop.get_data_set(name='ImaginaryDevice monitor for 10 s')

pp=qc.Plot()
data.publisher=pp

pp.add(data.timer,data.instrument_input1,subplot=0)
pp.add(data.timer,data.instrument_input2,subplot=1)

_=timeloop.run()

## More complicated experiments

In [ ]:
def get_conductance():
    return(current()/(12906*instrument.input2()))
conductance=qc.Parameter(name='conductance',label='G',unit='2e2/h',get_cmd=get_resistance)

station.set_measurement(instrument.input1,instrument.input2,current,conductance)
pp=qc.Plot()

for j in range(5):
    loop=qc.Loop(instrument.output1.sweep(-1,-0.5,num=21),delay=0.01)
    outerloop=qc.Loop(instrument.output2.sweep(-0.5,-0.3,num=21),delay=0.1).each(loop)
    data=outerloop.get_data_set(name='ImaginaryDevice Vout1(-1 -0.5)V Vout2(-0.5 -0.3) T=23.2K repeat={}'.format(j))
    data.publisher=pp
    pp.add(data.conductance,subplot=j)
    outerloop.run()